In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"/home/nisarg/Desktop/NLP OP/fake-news/train.csv")

In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [7]:
X = df.drop('label', axis = 1)
y = df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.4.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
voc_size = 5000

In [14]:
messages = X.copy()

In [15]:
messages.reset_index(inplace = True)

In [16]:
import re
import nltk
from nltk.corpus import stopwords

In [17]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)

    corpus.append(review)

In [18]:
# corpus

In [19]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
# onehot_repr

In [20]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen=sent_length)
# print(embedded_docs)

In [21]:
len(embedded_docs)

18285

In [22]:
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs), y.shape

(18285, (18285,))

In [24]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [25]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state=42)

In [28]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 17ms/step - loss: 0.4819 - accuracy: 0.7445 - val_loss: 0.1938 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1337 - accuracy: 0.9493 - val_loss: 0.2044 - val_accuracy: 0.9228
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0871 - accuracy: 0.9704 - val_loss: 0.2245 - val_accuracy: 0.9198
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0450 - accuracy: 0.9849 - val_loss: 0.2691 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0265 - accuracy: 0.9924 - val_loss: 0.3062 - val_accuracy: 0.9115
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0230 - accuracy: 0.9931 - val_loss: 0.3910 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0084 - accuracy: 0.9981 - val_loss: 0.4632 - val_accuracy: 0.9152

In [29]:
from tensorflow.keras.layers import Dropout
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [34]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 15ms/step - loss: 0.4879 - accuracy: 0.7694 - val_loss: 0.2052 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1630 - accuracy: 0.9356 - val_loss: 0.1944 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1078 - accuracy: 0.9601 - val_loss: 0.2076 - val_accuracy: 0.9210
Epoch 4/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0731 - accuracy: 0.9749 - val_loss: 0.2415 - val_accuracy: 0.9185
Epoch 5/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0497 - accuracy: 0.9833 - val_loss: 0.2979 - val_accuracy: 0.9173
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0361 - accuracy: 0.9883 - val_loss: 0.3542 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0290 - accuracy: 0.9909 - val_loss: 0.3973 - val_accuracy: 0.9173

In [35]:
y_pred = model.predict_classes(X_test)

/home/nisarg/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test, y_pred)

array([[3178,  241],
       [ 333, 2283]])

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9048881524440763